<center>
<img src="images/scaling-xgboost.svg" width="60%">
<\center> 

# Scaling XGBoost with Dask and Coiled

This notebook walks through training an [XGBoost](https://xgboost.readthedocs.io/en/latest/) model locally on a small dataset and then using [Dask](https://dask.org/) and [Coiled](https://coiled.io/) to scale out to the cloud and run XGBoost on a larger-than-memory dataset.

# Local XGBoost

[XGBoost](https://xgboost.readthedocs.io/en/latest/) is a popular library for training gradient boosted supervised machine learning models. 

## Load our dataset

The first step towards training our model is to load our dataset. We'll use the [Higgs dataset](https://archive.ics.uci.edu/ml/datasets/HIGGS), which is available on Amazon S3. The dataset is composed of 11 million simulated particle collisions, each of which is described by 28 real-valued, features and a binary label indicating which class the sample belongs to (i.e. whether the sample represents a signal or background event). To start, we'll load only a sample of the dataset (just over 175 thousand samples) and process the full datset in the next section.

In [1]:
import pandas as pd

# Load a single CSV file
df = pd.read_csv("s3://coiled-data/higgs/higgs-00.csv")
df

,labels,lepton_pT,lepton_eta,lepton_phi,missing_energy_magnitude,missing_energy_phi,jet_1_pt,jet_1_eta,jet_1_phi,jet_1_b-tag,...,jet_4_eta,jet_4_phi,jet_4_b-tag,m_jj,m_jjj,m_lv,m_jlv,m_bb,m_wbb,m_wwbb
0,1,0.869293,-0.635082,0.225690,0.327470,-0.689993,0.754202,-0.248573,-1.092064,0.000000,...,-0.010455,-0.045767,3.101961,1.353760,0.979563,0.978076,0.920005,0.721657,0.988751,0.876678
1,1,0.907542,0.329147,0.359412,1.497970,-0.313010,1.095531,-0.557525,-1.588230,2.173076,...,-1.138930,-0.000819,0.000000,0.302220,0.833048,0.985700,0.978098,0.779732,0.992356,0.798343
2,1,0.798835,1.470639,-1.635975,0.453773,0.425629,1.104875,1.282322,1.381664,0.000000,...,1.128848,0.900461,0.000000,0.909753,1.108330,0.985692,0.951331,0.803252,0.865924,0.780118
3,0,1.344385,-0.876626,0.935913,1.992050,0.882454,1.786066,-1.646778,-0.942383,0.000000,...,-0.678379,-1.360356,0.000000,0.946652,1.028704,0.998656,0.728281,0.869200,1.026736,0.957904
4,1,1.105009,0.321356,1.522401,0.882808,-1.205349,0.681466,-1.070464,-0.921871,0.000000,...,-0.373566,0.113041,0.000000,0.755856,1.361057,0.986610,0.838085,1.133295,0.872245,0.808487
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175216,1,0.556165,-1.221411,1.382576,0.485715,-1.048621,1.108081,-0.118853,1.646101,2.173076,...,1.265431,0.291167,0.000000,1.263402,1.034347,0.986912,0.733762,0.899887,1.118665,0.932763
175217,0,0.868561,-0.473403,0.530864,2.481327,0.981628,0.547078,-0.971441,-1.570490,0.000000,...,0.031187,-0.304356,1.550981,1.298022,0.841658,0.992392,1.120417,0.207082,0.781515,0.877750
175218,1,0.630832,-1.620738,-0.082360,0.866506,-1.247463,0.568423,-0.256495,-1.292748,2.173076,...,-2.413150,1.378795,0.000000,0.628195,0.764134,0.985883,0.711534,0.843467,0.805140,0.720537
175219,1,0.904980,-1.086029,-0.274342,1.158548,-1.258253,0.515474,-1.929983,0.133211,0.000000,...,0.377641,1.184021,0.000000,0.880999,0.949600,1.098452,0.930461,0.781536,0.818147,0.779077


Next, we can separate our classification label and training features and then use Scikit-learn's `sklearn.model_selection.train_test_split` function to partition the dataset into training and testing samples.

In [2]:
X, y = df.iloc[:, 1:], df["labels"]

In [3]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

To use XGBoost, we'll need to construct `xgboost.DMatrix` objects for both our training and testing datasets -- these are the internal data structures XGBoost uses to manage dataset features and targets. However, since XGBoost plays well with libaries like NumPy and Pandas, we can simply pass our training and testing datasets directly to `xgboost.DMatrix(...)`.

In [4]:
import xgboost

In [5]:
dtrain = xgboost.DMatrix(X_train, y_train)
dtest = xgboost.DMatrix(X_test, y_test)

Next we'll define the set of hyperparameters we want to use for our XGBoost model and train the model!

In [6]:
params = {'objective': 'binary:logistic',
          'max_depth': 3,
          'min_child_weight': 0.5}

In [7]:
bst = xgboost.train(params, dtrain, num_boost_round=3)

Now that our model has been trained, we can use it to make predictions on the testing dataset which was _not_ used to train the model.

In [8]:
y_pred = bst.predict(dtest)

In [9]:
y_pred

array([0.42195904, 0.5599851 , 0.4672076 , ..., 0.34625873, 0.28058735,
       0.4672076 ], dtype=float32)

To get a sense for the quality of these predictions we can compute and plot a [receiver operating characteristic (ROC) curve](https://en.wikipedia.org/wiki/Receiver_operating_characteristic) of our model's predictions, which compares the predicted output from our model with the known labels to calculate the true postive rate vs. false positive rate.

In [10]:
from sklearn.metrics import roc_curve

fpr, tpr, _ = roc_curve(y_test, y_pred)

In [11]:
from sklearn.metrics import auc
from bokeh.io import output_notebook
from bokeh.plotting import figure, output_file, show
output_notebook()

p = figure(title="ROC Curve",
           x_axis_label="False Positive Rate",
           y_axis_label="True Positive Rate",
           tools="pan,wheel_zoom,box_zoom,reset,box_select",
           tooltips=[("(x,y)", "($x, $y)")])

p.line(fpr, tpr, legend_label="ROC Curve (area = {:.2f})".format(auc(fpr, tpr)), line_width=2)
p.line([0, 1], [0, 1], line_width=2, color="black", line_dash="dashed")
show(p)

Loading BokehJS ...

# Scaling with Dask and Coiled

In the previous section, we trained a model with a subset of the full Higgs dataset. In this section, we will use the full dataset with 11 million samples! With this increased number of samples, the dataset may not fit comfortably into memory on a personal laptop. So we'll use Dask and Coiled to expand our compute resources to the cloud to enable us to work with this larger datset.

### Create a Dask cluster on AWS with Coiled

Let's create a Coiled cluster using the `coiled-examples/xgboost` cluster configuration, which has Dask, XGBoost, Scikit-learn, and other relavant packages installed, and then connect a `dask.distributed.Client` to our cluster so we can begin to submit tasks to the cluster.

In [12]:
import coiled
import dask.distributed

cluster = coiled.Cluster(configuration="coiled-examples/xgboost")
client = dask.distributed.Client(cluster)
client

Client Scheduler: tls://ec2-18-217-215-35.us-east-2.compute.amazonaws.com:8786 Dashboard: http://ec2-18-217-215-35.us-east-2.compute.amazonaws.com:8787/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


Be sure to click te "Dashboard:" link above to view the diagnostics dashboard for your cluster. This will show what tasks are currently being processed, how much memory your workers are using, etc.

### Load full dataset

Dask's `read_csv` functions makes it easy to read in all the CSV files in the dataset.

In [13]:
import dask.dataframe as dd

# Load the entire dataset using Dask
ddf = dd.read_csv("s3://coiled-data/higgs/higgs-*.csv")
ddf

,labels,lepton_pT,lepton_eta,lepton_phi,missing_energy_magnitude,missing_energy_phi,jet_1_pt,jet_1_eta,jet_1_phi,jet_1_b-tag,jet_2_pt,jet_2_eta,jet_2_phi,jet_2_b-tag,jet_3_pt,jet_3_eta,jet_3_phi,jet_3_b-tag,jet_4_pt,jet_4_eta,jet_4_phi,jet_4_b-tag,m_jj,m_jjj,m_lv,m_jlv,m_bb,m_wbb,m_wwbb
npartitions=126,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


Dask's machine learning library, [Dask-ML](https://ml.dask.org/), mimics Scikit-learn's API, providing scalable versions of functions of `sklearn.datasets.make_classification` and `sklearn.model_selection.train_test_split` that are designed to work with Dask Arrays and DataFrames in larger-than-memory settings.

Let's use Dask-ML to generate a similar classification dataset as before, but now with 100 million samples.

In [14]:
from dask_ml.model_selection import train_test_split

X, y = ddf.iloc[:, 1:], ddf["labels"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

/Users/james/miniforge3/envs/coiled-coiled-examples-xgboost/lib/python3.8/site-packages/dask_ml/model_selection/_split.py:462: FutureWarning: The default value for 'shuffle' must be specified when splitting DataFrames. In the future DataFrames will automatically be shuffled within blocks prior to splitting. Specify 'shuffle=True' to adopt the future behavior now, or 'shuffle=False' to retain the previous behavior.
  warnings.warn(


Next we'll [persist our training and testing datasets](https://distributed.dask.org/en/latest/memory.html#persisting-collections) into distributed memory to avoid any unnecessary re-computations.

In [15]:
import dask

X_train, X_test, y_train, y_test = dask.persist(X_train, X_test, y_train, y_test)

In [16]:
X_train

,lepton_pT,lepton_eta,lepton_phi,missing_energy_magnitude,missing_energy_phi,jet_1_pt,jet_1_eta,jet_1_phi,jet_1_b-tag,jet_2_pt,jet_2_eta,jet_2_phi,jet_2_b-tag,jet_3_pt,jet_3_eta,jet_3_phi,jet_3_b-tag,jet_4_pt,jet_4_eta,jet_4_phi,jet_4_b-tag,m_jj,m_jjj,m_lv,m_jlv,m_bb,m_wbb,m_wwbb
npartitions=126,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


To do distributed training of an XGBoost model, we'll use the [Dask-XGBoost](https://github.com/dask/dask-xgboost) package which mirrors XGBoost's interface but works with Dask Arrays and DataFrames.

In [17]:
import dask_xgboost

In [18]:
bst = dask_xgboost.train(client, params, X_train, y_train, num_boost_round=3)

Finally, we can again compute and plot the ROC curve for this model's predictions.

In [19]:
y_pred = dask_xgboost.predict(client, bst, X_test)

In [20]:
y_test, y_pred = dask.compute(y_test, y_pred)
fpr, tpr, _ = roc_curve(y_test, y_pred)

In [21]:
p = figure(title="ROC Curve",
           x_axis_label="False Positive Rate",
           y_axis_label="True Positive Rate",
           tools="pan,wheel_zoom,box_zoom,reset,box_select",
           tooltips=[("(x,y)", "($x, $y)")])

p.line(fpr, tpr, legend_label="ROC Curve (area = {:.2f})".format(auc(fpr, tpr)), line_width=2)
p.line([0, 1], [0, 1], line_width=2, color="black", line_dash="dashed")
show(p)

Voilà! Congratulations on training a boosted decision tree in the cloud from JupyterLab